In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
cosine_sim_feaure.gz
sample.rtf
sample_submission.csv
test.csv
train.csv



## Extract, Transform, Load

In [2]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    # Remove missing values and duplicates from training data
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")
# df_train, df_test = train_test_split(df, test_size = 0.10)
# df_train = df_train.reset_index(drop=True)
# df_train = df_train.reset_index()
# df_train = df_train.rename(columns = {'index':'mat_idx'})

df_submit = read_data("input/test.csv")
df_submit = df_submit.reset_index(drop=True)
df_submit = df_submit.reset_index()
df_submit = df_submit.rename(columns = {'index': 'mat_idx'})


# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404288, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345790, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Bag of Words helper

In [3]:
print("Creating the bag of words feature")
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

# Use this as "stop_words" arg if include stopwords
# stopwords.words('english')

# Initialize CountVectorizer
# For now assume stop-words are not discounted. No terms are thresholded. 
vectorizer = CountVectorizer(analyzer = stemmed_words,   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             ngram_range = (1,2), \
                             max_features = 100) 


Creating the bag of words feature


## Generate Test Set Features

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# cos_feature_vec = np.zeros((df_train.shape[0],1))

def gen_cosine_sim(s1, s2, vectorizer):
    try:
        features = vectorizer.fit_transform([s1, s2])
        similarities = cosine_similarity(features)
        return similarities[0,1]
#         cos_feature_vec[idx,0] = similarities[0,1]
    except:
        return 0.5

# for idx, row in df_train.iterrows():
#     if (not idx % 100000):
#         print("Generated feature for %d entries" % (idx))
#     try:
#         features = vectorizer.fit_transform([row['question1'], row['question2']])
#         similarities = cosine_similarity(features)
#         cos_feature_vec[idx,0] = similarities[0,1]
#     except:
#         print(row['question1'])
#         print(row['question2'])

In [5]:
# Optionally write the cosine similarity feature vector to compressed file for storage
# np.savetxt("input/cosine_sim_feaure.gz",cos_feature_vec)

## Building the Model

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

df_train['cos_sim'] = df_train.apply(lambda row: gen_cosine_sim(row['question1'],row['question2'],vectorizer), axis=1)

# cos_feature_vec.T[0].tolist()

print("Cosine Similarity Feature Generated...")

scaler = MinMaxScaler().fit(df_train[['cos_sim']])
X = scaler.transform(df_train[['cos_sim']])

y = df_train['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

Cosine Similarity Feature Generated...


((323430, 1), (80858, 1), (323430,), (80858,))

## Cross-Validation

In [14]:
clf = LogisticRegression()
grid = {
    'C': [1e-6, 1e-3, 1e0],
    'penalty': ['l1', 'l2']
}
cv = GridSearchCV(clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.3s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [1e-06, 0.001, 1.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=1)

In [15]:
for i in range(1, len(cv.cv_results_['params'])+1):
    rank = cv.cv_results_['rank_test_score'][i-1]
    s = cv.cv_results_['mean_test_score'][i-1]
    sd = cv.cv_results_['std_test_score'][i-1]
    params = cv.cv_results_['params'][i-1]
    print("{0}. Mean validation neg log loss: {1:.3f} (std: {2:.3f}) - {3}".format(
        rank,
        s,
        sd,
        params
    ))

6. Mean validation neg log loss: -0.693 (std: 0.000) - {'C': 1e-06, 'penalty': 'l1'}
5. Mean validation neg log loss: -0.690 (std: 0.000) - {'C': 1e-06, 'penalty': 'l2'}
3. Mean validation neg log loss: -0.594 (std: 0.001) - {'C': 0.001, 'penalty': 'l1'}
4. Mean validation neg log loss: -0.600 (std: 0.001) - {'C': 0.001, 'penalty': 'l2'}
1. Mean validation neg log loss: -0.592 (std: 0.001) - {'C': 1.0, 'penalty': 'l1'}
2. Mean validation neg log loss: -0.592 (std: 0.001) - {'C': 1.0, 'penalty': 'l2'}


In [16]:
# Print the best cross-validation parameters

print(cv.best_params_)
print(cv.best_estimator_.coef_)

{'C': 1.0, 'penalty': 'l1'}
[[ 3.30629813]]


## ROC

In [17]:
colors = ['r', 'g', 'b', 'y', 'k', 'c', 'm', 'brown', 'r']
lw = 1
Cs = [1e-6, 1e-4, 1e0]

plt.figure(figsize=(12,8))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for different classifiers')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

labels = []
for idx, C in enumerate(Cs):
    clf = LogisticRegression(C = C)
    clf.fit(X_train, y_train)
    print("C: {}, parameters {} and intercept {}".format(C, clf.coef_, clf.intercept_))
    fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=colors[idx])
    labels.append("C: {}, AUC = {}".format(C, np.round(roc_auc, 4)))

plt.legend(['random AUC = 0.5'] + labels)

C: 1e-06, parameters [[-0.00691736]] and intercept [-0.03892372]
C: 0.0001, parameters [[ 0.65118959]] and intercept [-0.78502616]
C: 1.0, parameters [[ 3.30586098]] and intercept [-2.39640651]


## Precision-Recall Curve

In [18]:
pr, re, _ = precision_recall_curve(y_test, cv.best_estimator_.predict_proba(X_test)[:,1])
plt.figure(figsize=(12,8))
plt.plot(re, pr)
plt.title('PR Curve (AUC {})'.format(auc(re, pr)))
plt.xlabel('Recall')
plt.ylabel('Precision')

## Submission

In [ ]:
df_submit['cos_sim'] = df_submit.apply(lambda row: gen_cosine_sim(row['question1'],row['question2'],vectorizer), axis=1)

df_submit.head()

In [6]:
X

NameError: name 'X' is not defined